In [1]:
#Read tif volume
import numpy as np
import tifffile as tiff
import os
import sys
import matplotlib.pyplot as plt

def read_tif_volume(tif_file):
    #Read tif volume
    volume = tiff.imread(tif_file)
    return volume

def write_tif_volume(volume, tif_file):
    tiff.imwrite(tif_file, volume)


In [2]:
skeleton = read_tif_volume('outputs/output_skeleton_volume.tif')

print(f"Number of pixels in the volume: {np.size(skeleton)}")
print(f"Number of pixels set to True: {np.sum(skeleton==True)}")
print(f"Number of pixels set to False: {np.sum(skeleton == False, dtype=np.int64)}")


Number of pixels in the volume: 2666222100
Number of pixels set to True: 133044
Number of pixels set to False: 2666089056


In [3]:
sk_copy = skeleton.copy()

from skan import Skeleton, summarize

branch_data = summarize(Skeleton(sk_copy))

branch_type_0 =  branch_data[branch_data['branch-type'] == 0]
branch_type_1 =  branch_data[branch_data['branch-type'] == 1]
branch_type_2 =  branch_data[branch_data['branch-type'] == 2]
branch_type_3 =  branch_data[branch_data['branch-type'] == 3]

print(f"Number of branch type 0: {len(branch_type_0)}")
print(f"Number of branch type 1: {len(branch_type_1)}")
print(f"Number of branch type 2: {len(branch_type_2)}")
print(f"Number of branch type 3: {len(branch_type_3)}")



Number of branch type 0: 47
Number of branch type 1: 4486
Number of branch type 2: 5268
Number of branch type 3: 17


In [4]:
# Iterate over the short branches and set the corresponding pixels to 0 in the binary volume
for _, row in branch_type_3.iterrows():
    src_z, src_y, src_x = int(row['image-coord-src-0']), int(row['image-coord-src-1']), int(row['image-coord-src-2'])
    dst_z, dst_y, dst_x = int(row['image-coord-dst-0']), int(row['image-coord-dst-1']), int(row['image-coord-dst-2'])
    print(f"Source pixel: {src_z}, {src_y}, {src_x}")
    print(f"Destination pixel: {dst_z}, {dst_y}, {dst_x}")

    # Set the source and destination pixels to 0
    sk_copy[src_z, src_y, src_x] = 0
    sk_copy[dst_z, dst_y, dst_x] = 0

print("Branch type 3 removed.")

print(f"Number of pixels in the volume: {np.size(sk_copy)}")
print(f"Number of pixels set to True: {np.sum(sk_copy==True)}")
print(f"Number of pixels set to False: {np.sum(sk_copy == False, dtype=np.int64)}")


Source pixel: 248, 1091, 596
Destination pixel: 248, 1091, 596
Source pixel: 334, 990, 1155
Destination pixel: 334, 990, 1155
Source pixel: 440, 914, 1185
Destination pixel: 440, 914, 1185
Source pixel: 469, 955, 636
Destination pixel: 469, 955, 636
Source pixel: 519, 931, 641
Destination pixel: 519, 931, 641
Source pixel: 552, 837, 1018
Destination pixel: 552, 837, 1018
Source pixel: 580, 1169, 772
Destination pixel: 580, 1169, 772
Source pixel: 644, 992, 1146
Destination pixel: 644, 992, 1146
Source pixel: 645, 1108, 759
Destination pixel: 645, 1108, 759
Source pixel: 670, 764, 1123
Destination pixel: 670, 764, 1123
Source pixel: 693, 840, 1034
Destination pixel: 693, 840, 1034
Source pixel: 737, 739, 1025
Destination pixel: 737, 739, 1025
Source pixel: 746, 699, 521
Destination pixel: 746, 699, 521
Source pixel: 786, 857, 898
Destination pixel: 786, 857, 898
Source pixel: 796, 374, 601
Destination pixel: 796, 374, 601
Source pixel: 796, 939, 760
Destination pixel: 796, 939, 760
Sour

In [5]:
import numpy as np

def bresenham_3d(x1, y1, z1, x2, y2, z2):
    """Bresenham's line algorithm for 3D."""
    points = []
    dx = abs(x2 - x1)
    dy = abs(y2 - y1)
    dz = abs(z2 - z1)
    xs = 1 if x2 > x1 else -1
    ys = 1 if y2 > y1 else -1
    zs = 1 if z2 > z1 else -1

    # Driving axis is X-axis
    if dx >= dy and dx >= dz:
        p1 = 2 * dy - dx
        p2 = 2 * dz - dx
        while x1 != x2:
            points.append((x1, y1, z1))
            x1 += xs
            if p1 >= 0:
                y1 += ys
                p1 -= 2 * dx
            if p2 >= 0:
                z1 += zs
                p2 -= 2 * dx
            p1 += 2 * dy
            p2 += 2 * dz

    # Driving axis is Y-axis
    elif dy >= dx and dy >= dz:
        p1 = 2 * dx - dy
        p2 = 2 * dz - dy
        while y1 != y2:
            points.append((x1, y1, z1))
            y1 += ys
            if p1 >= 0:
                x1 += xs
                p1 -= 2 * dy
            if p2 >= 0:
                z1 += zs
                p2 -= 2 * dy
            p1 += 2 * dx
            p2 += 2 * dz

    # Driving axis is Z-axis
    else:
        p1 = 2 * dy - dz
        p2 = 2 * dx - dz
        while z1 != z2:
            points.append((x1, y1, z1))
            z1 += zs
            if p1 >= 0:
                y1 += ys
                p1 -= 2 * dz
            if p2 >= 0:
                x1 += xs
                p2 -= 2 * dz
            p1 += 2 * dy
            p2 += 2 * dx

    points.append((x2, y2, z2))
    return points

# Iterate over the short branches and set the corresponding pixels to 0 in the binary volume
for _, row in branch_type_0.iterrows():
    src_z, src_y, src_x = int(row['image-coord-src-0']), int(row['image-coord-src-1']), int(row['image-coord-src-2'])
    dst_z, dst_y, dst_x = int(row['image-coord-dst-0']), int(row['image-coord-dst-1']), int(row['image-coord-dst-2'])


    # Get all the pixels between source and destination
    branch_pixels = bresenham_3d(src_x, src_y, src_z, dst_x, dst_y, dst_z)
    print(f"Number of pixels in the branch: {len(branch_pixels)}")



    # Set all the pixels in the branch to 0
    for x, y, z in branch_pixels:
        sk_copy[z, y, x] = 0

print("Branch type 3 removed.")

print(f"Number of pixels in the volume: {np.size(sk_copy)}")
print(f"Number of pixels set to True: {np.sum(sk_copy==True)}")
print(f"Number of pixels set to False: {np.sum(sk_copy == False, dtype=np.int64)}")

Number of pixels in the branch: 15
Number of pixels in the branch: 10
Number of pixels in the branch: 2
Number of pixels in the branch: 2
Number of pixels in the branch: 3
Number of pixels in the branch: 3
Number of pixels in the branch: 2
Number of pixels in the branch: 3
Number of pixels in the branch: 3
Number of pixels in the branch: 4
Number of pixels in the branch: 14
Number of pixels in the branch: 5
Number of pixels in the branch: 4
Number of pixels in the branch: 3
Number of pixels in the branch: 3
Number of pixels in the branch: 3
Number of pixels in the branch: 2
Number of pixels in the branch: 2
Number of pixels in the branch: 2
Number of pixels in the branch: 2
Number of pixels in the branch: 2
Number of pixels in the branch: 2
Number of pixels in the branch: 8
Number of pixels in the branch: 23
Number of pixels in the branch: 2
Number of pixels in the branch: 11
Number of pixels in the branch: 22
Number of pixels in the branch: 9
Number of pixels in the branch: 2
Number o

In [6]:
pixels_to_delete = []
i = 0
# Iterate over the short branches and set the corresponding pixels to 0 in the binary volume
for _, row in branch_type_1.iterrows():
    src_z, src_y, src_x = int(row['image-coord-src-0']), int(row['image-coord-src-1']), int(row['image-coord-src-2'])
    dst_z, dst_y, dst_x = int(row['image-coord-dst-0']), int(row['image-coord-dst-1']), int(row['image-coord-dst-2'])

    # Get all the pixels between source and destination
    branch_pixels = bresenham_3d(src_x, src_y, src_z, dst_x, dst_y, dst_z)
    print(f"Number of pixels in the branch: {len(branch_pixels)}")
    i += 1
    print(f"Branch number: {i}")

    # Save the pixels to delete
    pixels_to_delete.append(branch_pixels)

# Mean value of pixels in a branch
mean_branch_pixels = np.mean([len(branch) for branch in pixels_to_delete])
print(f"Mean number of pixels in a branch: {mean_branch_pixels}")
    

Number of pixels in the branch: 33
Branch number: 1
Number of pixels in the branch: 23
Branch number: 2
Number of pixels in the branch: 29
Branch number: 3
Number of pixels in the branch: 18
Branch number: 4
Number of pixels in the branch: 12
Branch number: 5
Number of pixels in the branch: 9
Branch number: 6
Number of pixels in the branch: 4
Branch number: 7
Number of pixels in the branch: 8
Branch number: 8
Number of pixels in the branch: 17
Branch number: 9
Number of pixels in the branch: 21
Branch number: 10
Number of pixels in the branch: 34
Branch number: 11
Number of pixels in the branch: 31
Branch number: 12
Number of pixels in the branch: 12
Branch number: 13
Number of pixels in the branch: 21
Branch number: 14
Number of pixels in the branch: 25
Branch number: 15
Number of pixels in the branch: 32
Branch number: 16
Number of pixels in the branch: 16
Branch number: 17
Number of pixels in the branch: 49
Branch number: 18
Number of pixels in the branch: 11
Branch number: 19
Numbe

In [7]:
# Iterate over the short branches and set the corresponding pixels to 0 in the binary volume
for branch in pixels_to_delete:
    if len(branch) < 10:
        for x, y, z in branch:
            sk_copy[z, y, x] = 0

print(f"Number of pixels in the volume: {np.size(sk_copy)}")
print(f"Number of pixels set to True: {np.sum(sk_copy==True)}")
print(f"Number of pixels set to False: {np.sum(sk_copy == False, dtype=np.int64)}")

Number of pixels in the volume: 2666222100
Number of pixels set to True: 126702
Number of pixels set to False: 2666095398


In [8]:
write_tif_volume(sk_copy, 'outputs/skeleton_cleaned.tif')

In [9]:
branch_data = summarize(Skeleton(sk_copy))

branch_type_0 =  branch_data[branch_data['branch-type'] == 0]
branch_type_1 =  branch_data[branch_data['branch-type'] == 1]
branch_type_2 =  branch_data[branch_data['branch-type'] == 2]
branch_type_3 =  branch_data[branch_data['branch-type'] == 3]

print(f"Number of branch type 0: {len(branch_type_0)}")
print(f"Number of branch type 1: {len(branch_type_1)}")
print(f"Number of branch type 2: {len(branch_type_2)}")
print(f"Number of branch type 3: {len(branch_type_3)}")

Number of branch type 0: 1236
Number of branch type 1: 3071
Number of branch type 2: 3223
Number of branch type 3: 41
